libraries 


In [1]:
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from datetime import timedelta, date,datetime
import numpy as np
import threading

Drive Mount


In [2]:
import glob
from google.colab import drive
drive.mount('/content/drive') #mount with drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Trend computation


In [3]:
class TrendComputation:

  def daterange(self,start_date, end_date):    # to generate dates  between start date and end date 
     self.start_date=start_date
     self.end_date=end_date 
     for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)



  def helper(self,source,start_date,end_date,numOfDays):   #to store results from trend compute to desired place
        self.source=source
        self.start_date = start_date
        self.end_date = end_date
        self. numOfDays=numOfDays
        #print('days[[[[[[[[[[[[[',numOfDays)
        i=0
        temp1=1   # to find trends according to input num of days e.f 1 4,3 7 
        for single_date in self.daterange(start_date, end_date):
           if(temp1%(numOfDays)==0):                    # for trends time i.e daily , wekly, monthly etc
            #  t1=TrendComputation()   #object of class
              aa=single_date.strftime("%Y-%m-%d")       #date to this formate for unique/symmetic pattern
              bb=end_date.strftime("%Y-%m-%d")
              #print('dateeeeeeeeeeeeeeeeeeee',aa)    #check if date converted into right formte or not
              #print('dateeeeeeeeeeeeeeeeeeee',bb)     #check if date converted into right formte or not
              trends=[]
              freqArray,tdidfArray,trendsArray=self.readData(source,aa,bb,numOfDays)   # pass start date source and end date to compute trends and store to write in desired path
              #print('================================',trendsArray,"--------------------------------------------------------",single_date)
              dict = {'trends': trendsArray, 'tfidfvalue':freqArray ,'frequency':tdidfArray}   # write dict to db
              df = pd.DataFrame(dict) 
              #print('i=>',i)
              if(source!='all'):  #write outputs of dict to file
                 with open('/content/drive/MyDrive/Dataset_NameEntityComputer/Trends/'+ source +'/'+ str(single_date)  +'----'+ str(numOfDays) +'.csv', 'a',) as f:
                   df.to_csv(f,header=f.tell()==0) 
                   print(source,'-----> ',single_date)
              else:
                 with open('/content/drive/MyDrive/Dataset_NameEntityComputer/Trends/'+ source +'/'+ str(single_date)  +'----'+ str(numOfDays) +'.csv', 'a',) as f:
                   df.to_csv(f,header=f.tell()==0)
                   print(source,'======>',single_date)                 
           temp1=temp1+1





  def readData(self,source,start_date,end_date,days): 
     mylist = [ "sputnik", "xinhua","tass","scmp","NYT","dawn","tribune","toi"]   #if ant to calculate trends in combinations all possible sources
     self.source=source
     self.start_date=start_date
     self.end_date=end_date
     self.days=days
     start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
     end_date = datetime.strptime(end_date, '%Y-%m-%d').date()     
     tempcorpus = []  # make corpus for td idf
     #tempdata=""
     tempdaycounter=0  # counter to comapre it with dates i.e 1,3 5 etc
     #tempdata2=""
     #tempdata3=""
     if(source=='all'):    # code for trends in combination multiple sources
        for ii in range(len(mylist)):   # making corpus
           for single_date in self.daterange(start_date, end_date):
               if(days==tempdaycounter):   #if num of days satisfy it will break and back to function e.g if day is 7 it will add data of 7 days and break
                   #print(start_date, mylist[ii] ,end_date)
                   break;
               else:
                 tempdaycounter=tempdaycounter+1   
               date=(single_date.strftime("%Y-%m-%d"))
               #print(mylist[ii] ,'------------------------------',single_date.strftime("%Y-%m-%d"))
               col_list = ["NER"]
               try:

                 text=pd.read_csv('/content/drive/MyDrive/Dataset_NameEntityComputer/'+ mylist[ii] +'/'+ date +'.csv' , usecols=col_list)
                 for i in range(len(text)):
                   temp1 = ' '.join([str(elem) for elem in text.iloc[i]])
                   tempcorpus.append(temp1) 
               except IOError:
                print('data for ',date,' not found')

        txt1= pd.DataFrame(tempcorpus)         
        tempdata=self.removeNoise(txt1)
        #print("=====================")
        #tempdata1 =' '.join([str(elem) for elem in tempdata]) 
        tempdata2=self.corpusadd(tempdata)   # add all txts to generate corpus
        tempdata3,tempdata4,tempdata5=self.tfidf(tempdata2)      # calculate tf-idf for given corpus
     else:                                             # code for trends in combination single sources
        for single_date in self.daterange(start_date, end_date):  
            if(days==tempdaycounter):    #if num of days satisfy it will break and back to function e.g if day is 7 it will add data of 7 days and break
                #print(start_date, source ,end_date)
                break;
            else:
              tempdaycounter=tempdaycounter+1    
            date=(single_date.strftime("%Y-%m-%d"))
            #print(source,'-------------------------------------',single_date.strftime("%Y-%m-%d"))
            col_list = ["NER"]  #read ners only from csv
            try:

              text=pd.read_csv('/content/drive/MyDrive/Dataset_NameEntityComputer/'+ source +'/'+ date +'.csv' , usecols=col_list)
              for i in range(len(text)):
                #print(source,"sssssss",date)
                temp1 = ' '.join([str(elem) for elem in text.iloc[i]])
                tempcorpus.append(temp1) 

            except IOError:
              print('data for ',date,' not found')
                 
        txt1= pd.DataFrame(tempcorpus)         #store data in pandas dataframe
        tempdata=self.removeNoise(txt1)        # remove some noise i.e \n
        #print("=====================")
        #tempdata1 =' '.join([str(elem) for elem in tempdata]) 
        tempdata2=self.corpusadd(tempdata)    #add all data to one  corpus
        tempdata3,tempdata4,tempdata5=self.tfidf(tempdata2)  #calculate tf idf for thaat data
     return tempdata3 ,tempdata4 ,tempdata5

  #def sizeData(self,)
    # size=len(txt1)
    # return size

  def removeNoise(self,text):   # if there is any noise in our case \n is used earlier for name entities trends so remove it
    self.text=text
    txt2=text.replace(r'\n', ',', regex=True)
    return txt2

  def corpusadd(self,text):    # add corpus for selected time to generate tfidf for it , first we need to make corpus 
    self.text=text
    corpus1 = []
    for i in range(len(text)):
      temp1 = ' '.join([str(elem) for elem in text.iloc[i]])
      corpus1.append(temp1)
    return corpus1  

  def tfidf(self,corpus):   # caculate term frequency and inverse document frequency
    self.corpus=corpus
    re_exp = r","
    vectorizer = TfidfVectorizer(tokenizer=lambda text: re.split(re_exp,text) ,lowercase=False)  # lower case true to make U.S to us i.e both count same not different

    X = vectorizer.fit_transform(corpus)  

    counter=np.zeros(len(vectorizer.get_feature_names())) #  for tf idf values
    counterFrequency=np.zeros(len(vectorizer.get_feature_names())) #for frequemcy of ners
    for i in range(len(corpus)):
      tempArray=X[i].toarray()  #it will store unique ners 
      for j in range(len(vectorizer.get_feature_names())):
        if(tempArray[0][j]!=0):
           counter[j]=counter[j]+ tempArray[0][j]  #can add late but adding tf idf value 
           counterFrequency[j]=counterFrequency[j]+ 1
    trends=[]   # for store trends freq
    trendstfidf=[] # for tf id value 
    temparray1=vectorizer.get_feature_names() # feature array having value greater than 1 are trends
    #print('frequency------------------',counterFrequency)
    #print('tfidf value ---------------',counter)
    #print('==========',temparray1)
    freq=[]  #  store freq in db
    tfidfvalue=[]  # store tf idfvalue
    n =len(counter)
    indices = (-counter).argsort()[:n]
    for i in range(len(indices)):
        j=indices[i]
        trends.append(temparray1[j])
        tfidfvalue.append(counter[j])
        freq.append(counterFrequency[j])
        #print("------",counter[j])    #debugging for tf idf value
    return tfidfvalue, freq ,trends  





Main body --Daily trends


In [4]:
start_date = date(2021,3,14)    #  start date from which treb=nds need to be calculated
end_date = date(2021,7,22)      # end date to which trends need to be calculated
numOfDays=1  # trends for how many number of days i.e weekly daily monthly etc

######################################daily###########
tass=TrendComputation()
toi=TrendComputation() #class object
nyt=TrendComputation()
dawn=TrendComputation()
tribune=TrendComputation() 
tass=TrendComputation()  #class object
scmp=TrendComputation()
sputnik=TrendComputation()
xinhua=TrendComputation()
all=TrendComputation()

t1=threading.Thread(target=tass.helper, args=('tass',start_date,end_date,numOfDays, ))
t2=threading.Thread(target=scmp.helper, args=('scmp',start_date,end_date,numOfDays, ))
t3=threading.Thread(target=xinhua.helper, args=('xinhua',start_date,end_date,numOfDays, ))
t4=threading.Thread(target=sputnik.helper, args=('sputnik',start_date,end_date,numOfDays, ))
t6=threading.Thread(target=nyt.helper, args=('NYT',start_date,end_date,numOfDays, ))
t5=threading.Thread(target=dawn.helper, args=('dawn',start_date,end_date,numOfDays, ))
t7=threading.Thread(target=tribune.helper, args=('tribune',start_date,end_date,numOfDays, ))
t8=threading.Thread(target=toi.helper, args=('toi',start_date,end_date,numOfDays, ))
t9=threading.Thread(target=all.helper, args=('all',start_date,end_date,numOfDays, ))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()
t7.start()
t8.start()
t9.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
t7.join()
t8.join()
t9.join()

scmp ----->  2021-03-14
tass ----->  NYT ----->  2021-03-14
2021-03-14
xinhua ----->  2021-03-14
scmp ----->  2021-03-15
all ======> 2021-03-14
NYT ----->  2021-03-15
tribune ----->  2021-03-14
xinhuadawn ----->  ----->  2021-03-15
 2021-03-14
sputnik ----->  2021-03-14
scmp ----->  2021-03-16
xinhua -----> NYT ----->  2021-03-16
 2021-03-16
toitribune ----->  2021-03-15
 ----->  2021-03-14
tass ----->  2021-03-15
scmp ----->  2021-03-17
dawn ----->  2021-03-15
xinhua ----->  2021-03-17
NYT ----->  2021-03-17
tribunescmp ----->  2021-03-18
 ----->  2021-03-16
NYT ----->  2021-03-18
xinhua ----->  2021-03-18
tass ----->  2021-03-16
scmp ----->  dawn2021-03-19
tribune ----->  2021-03-17
 ----->  2021-03-16
xinhua ----->  2021-03-19
NYT ----->  2021-03-19
toi ----->  2021-03-15
xinhua ----->  2021-03-20
tribune ----->  2021-03-18
NYT ----->  2021-03-20
dawn ----->  2021-03-17
all ======> 2021-03-15
scmp ----->  2021-03-20
xinhua ----->  2021-03-21
NYT ----->  2021-03-21
tribune ----->  20

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-95d9b90400a0>", line 27, in helper
    freqArray,tdidfArray,trendsArray=self.readData(source,aa,bb,numOfDays)   # pass start date source and end date to compute trends and store to write in desired path
  File "<ipython-input-3-95d9b90400a0>", line 111, in readData
    tempdata3,tempdata4,tempdata5=self.tfidf(tempdata2)  #calculate tf idf for thaat data
  File "<ipython-input-3-95d9b90400a0>", line 136, in tfidf
    X = vectorizer.fit_transform(corpus)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py", line 1859, in fit_transform
    X = super().fit_transform(raw_documents)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py", line 1220, in fit_t

tass ----->  2021-04-05
scmp ----->  2021-04-25
tribune ----->  2021-04-21
scmp ----->  2021-04-26
xinhua ----->  2021-05-03
dawn ----->  2021-04-14
tribune ----->  2021-04-22
xinhua ----->  2021-05-04
scmp ----->  2021-04-27
tass ----->  2021-04-06
toi dawn ----->  2021-03-30
----->  2021-04-15
xinhua ----->  2021-05-05
tribune ----->  2021-04-23
scmp ----->  2021-04-28
all ======> 2021-03-24
xinhua ----->  2021-05-06
dawn ----->  2021-04-16
tribune ----->  2021-04-24
xinhua ----->  2021-05-07
sputnik ----->  2021-03-24
tribune ----->  2021-04-25
xinhua ----->  2021-05-08
tass ----->  2021-04-07
dawn ----->  2021-04-17
scmp ----->  2021-04-29
xinhua ----->  2021-05-09
toi ----->  2021-03-31
tribune ----->  2021-04-26
xinhua ----->  2021-05-10
dawn ----->  2021-04-18
scmp ----->  2021-04-30
tass ----->  2021-04-08
xinhua ----->  2021-05-11
tribune ----->  2021-04-27
toi ----->  2021-04-01
scmp ----->  2021-05-01
dawn ----->  2021-04-19
xinhua ----->  2021-05-12
tribune ----->  2021-04-

Weekly


In [ ]:
start_date = date(2021,3,14)    #  start date from which treb=nds need to be calculated
end_date = date(2021,7,22)      # end date to which trends need to be calculated
numOfDays=7  # trends for how many number of days i.e weekly daily monthly etc

######################################weekly###########
tass=TrendComputation()
toi=TrendComputation() #class object
nyt=TrendComputation()
dawn=TrendComputation()
tribune=TrendComputation() 
tass=TrendComputation()  #class object
scmp=TrendComputation()
sputnik=TrendComputation()
xinhua=TrendComputation()
all=TrendComputation()

t1=threading.Thread(target=tass.helper, args=('tass',start_date,end_date,numOfDays, ))
t2=threading.Thread(target=scmp.helper, args=('scmp',start_date,end_date,numOfDays, ))
t3=threading.Thread(target=xinhua.helper, args=('xinhua',start_date,end_date,numOfDays, ))
t4=threading.Thread(target=sputnik.helper, args=('sputnik',start_date,end_date,numOfDays, ))
t6=threading.Thread(target=nyt.helper, args=('NYT',start_date,end_date,numOfDays, ))
t5=threading.Thread(target=dawn.helper, args=('dawn',start_date,end_date,numOfDays, ))
t7=threading.Thread(target=tribune.helper, args=('tribune',start_date,end_date,numOfDays, ))
t8=threading.Thread(target=toi.helper, args=('toi',start_date,end_date,numOfDays, ))
t9=threading.Thread(target=all.helper, args=('all',start_date,end_date,numOfDays, ))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()
t7.start()
t8.start()
t9.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
t7.join()
t8.join()
t9.join()

NYT ----->  2021-03-20
xinhua ----->  2021-03-20
NYT ----->  2021-03-27
tribune ----->  2021-03-20
NYT ----->  2021-04-03
NYT ----->  2021-04-10
xinhua ----->  2021-03-27


Monthly

In [ ]:
start_date = date(2021,3,14)    #  start date from which treb=nds need to be calculated
end_date = date(2021,7,22)      # end date to which trends need to be calculated
numOfDays=30  # trends for how many number of days i.e weekly daily monthly etc

######################################monthly###########
tass=TrendComputation()
toi=TrendComputation() #class object
nyt=TrendComputation()
dawn=TrendComputation()
tribune=TrendComputation() 
tass=TrendComputation()  #class object
scmp=TrendComputation()
sputnik=TrendComputation()
xinhua=TrendComputation()
all=TrendComputation()

t1=threading.Thread(target=tass.helper, args=('tass',start_date,end_date,numOfDays, ))
t2=threading.Thread(target=scmp.helper, args=('scmp',start_date,end_date,numOfDays, ))
t3=threading.Thread(target=xinhua.helper, args=('xinhua',start_date,end_date,numOfDays, ))
t4=threading.Thread(target=sputnik.helper, args=('sputnik',start_date,end_date,numOfDays, ))
t6=threading.Thread(target=nyt.helper, args=('NYT',start_date,end_date,numOfDays, ))
t5=threading.Thread(target=dawn.helper, args=('dawn',start_date,end_date,numOfDays, ))
t7=threading.Thread(target=tribune.helper, args=('tribune',start_date,end_date,numOfDays, ))
t8=threading.Thread(target=toi.helper, args=('toi',start_date,end_date,numOfDays, ))
t9=threading.Thread(target=all.helper, args=('all',start_date,end_date,numOfDays, ))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()
t7.start()
t8.start()
t9.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
t7.join()
t8.join()
t9.join()

In [ ]:
 lk
start_date = date(2021,3,14)    #  start date from which treb=nds need to be calculated
end_date = date(2021,7,22)      # end date to which trends need to be calculated
numOfDays=1   # trends for how many number of days i.e weekly daily monthly etc

######################################daily###########
source='tass' 
tass=TrendComputation()
tass.helper(source,start_date,end_date,numOfDays)


source='xinhua' 
xinhua=TrendComputation()
xinhua.helper(source,start_date,end_date,numOfDays)

source='sputnik' 
sputnik=TrendComputation()
sputnik.helper(source,start_date,end_date,numOfDays)

source='scmp' 
scmp=TrendComputation()
scmp.helper(source,start_date,end_date,numOfDays)

source='all' 
all=TrendComputation()
all.helper(source,start_date,end_date,numOfDays)

In [ ]:
start_date = date(2021,3,14)    #  start date from which treb=nds need to be calculated
end_date = date(2021,7,22)      # end date to which trends need to be calculated
numOfDays=7   # trends for how many number of days i.e weekly daily monthly etc

######################################daily###########
#source='tass' 
#tass=TrendComputation()
#tass.helper(source,start_date,end_date,numOfDays)


#source='xinhua' 
#xinhua=TrendComputation()
#xinhua.helper(source,start_date,end_date,numOfDays)

#source='sputnik' 
#sputnik=TrendComputation()
#sputnik.helper(source,start_date,end_date,numOfDays)

#source='scmp' 
#scmp=TrendComputation()
#scmp.helper(source,start_date,end_date,numOfDays)

#source='all' 
#all=TrendComputation()
#all.helper(source,start_date,end_date,numOfDays)

In [ ]:
# source='xinhua'             
# start_date = date(2021,4,1)    
# end_date = date(2021,5,9)    
# numOfDays=1   
# t2=TrendComputation()
# t2.helper(source,start_date,end_date,numOfDays)

In [ ]:
# source='scmp'             
# start_date = date(2021,4,1)    
# end_date = date(2021,5,9)    
# numOfDays=1   
# t2=TrendComputation()
# t2.helper(source,start_date,end_date,numOfDays)

In [ ]:
# source='tass'             
# start_date = date(2021,4,1)    
# end_date = date(2021,5,9)    
# numOfDays=1   
# t2=TrendComputation()
# t2.helper(source,start_date,end_date,numOfDays)